# Use CASH_OUT_Template Customized Notebook Template

In [1]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

import pandas as pd
import datetime as dt

In [10]:
#!pip freeze | grep pandas

pandas==2.0.3


In [2]:
#import seaborn as sns
import subprocess

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

In [3]:
from fosforml import register_model
import requests

In [4]:
# Import label encoder 
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from math import sqrt
import numpy as np

In [5]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()
df_original = sf_df.to_pandas()

In [15]:
# To read a specific dataset published from a snowflake connection
#df_original = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")
#df = snowflake.get_dataframe("ATM_TRANSACTION_MASTER_DATA")

In [6]:
to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME','LOCATION_TYPE','ATM_PLACEMENTS','ATM_ACCESSIBILITY','ATM_MAINTAINENANCE','ATM__REPLENISHMENT']
df.drop(to_drop, inplace=True, axis=1)

In [7]:
df.shape

(312417, 11)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
dtypes: float64(4), int8(1), object(6)
memory usage: 24.1+ MB


In [9]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY
0,2023-01-29,Y,N,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0
1,2023-01-30,Y,N,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0
2,2023-01-31,Y,N,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0
3,2023-02-01,Y,N,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0
4,2023-02-02,Y,N,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0


In [10]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY'],
      dtype='object')

In [11]:
df.isnull().sum()

DATE                       0
WEEKDAY_FLAG               0
HOLIDAY_FLAG               0
BANK_ID                    0
STATE                      0
BANK_NAME                  0
ATM_ID                     0
DISPENSED_AMOUNT           0
TOTAL_TRANSACTION_COUNT    0
DOWNTIME_IN_MINS           0
ATM_MAX_CAPACITY           0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  object 
 2   HOLIDAY_FLAG             312417 non-null  object 
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
dtypes: float64(4), int8(1), object(6)
memory usage: 24.1+ MB


In [13]:
df['DATE'] = pd.to_datetime(df['DATE'])
df['HOUR'] = df['DATE'].dt.hour
df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
df['QUARTER'] = df['DATE'].dt.quarter
df['MONTH'] = df['DATE'].dt.month
df['YEAR'] = df['DATE'].dt.year
df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
df['DAYOFMONTH'] = df['DATE'].dt.day
df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [14]:
df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})

In [15]:
df.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,ATM_MAX_CAPACITY,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,2023-01-29 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,225180.0,90.0,0.0,2860000.0,0,6,1,1,2023,29,29
1,2023-01-30 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,262440.0,117.0,0.0,2860000.0,0,0,1,1,2023,30,30
2,2023-01-31 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,561150.0,160.0,0.0,2860000.0,0,1,1,1,2023,31,31
3,2023-02-01 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,437220.0,129.0,0.0,2860000.0,0,2,1,2,2023,32,1
4,2023-02-02 00:00:00.000000,1,0,3,Maine,Citibank,TBH000274025,360900.0,119.0,0.0,2860000.0,0,3,1,2,2023,33,2


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312417 entries, 0 to 312416
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DATE                     312417 non-null  object 
 1   WEEKDAY_FLAG             312417 non-null  int64  
 2   HOLIDAY_FLAG             312417 non-null  int64  
 3   BANK_ID                  312417 non-null  int8   
 4   STATE                    312417 non-null  object 
 5   BANK_NAME                312417 non-null  object 
 6   ATM_ID                   312417 non-null  object 
 7   DISPENSED_AMOUNT         312417 non-null  float64
 8   TOTAL_TRANSACTION_COUNT  312417 non-null  float64
 9   DOWNTIME_IN_MINS         312417 non-null  float64
 10  ATM_MAX_CAPACITY         312417 non-null  float64
 11  HOUR                     312417 non-null  int32  
 12  DAYOFWEEK                312417 non-null  int32  
 13  QUARTER                  312417 non-null  int32  
 14  MONT

In [17]:
# how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['STATE']= label_encoder.fit_transform(df['STATE'])

In [18]:
# how to understand word labels. 
atm_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['ATM_ID']= atm_encoder.fit_transform(df['ATM_ID'])

In [19]:
df.drop(['DATE','BANK_NAME','TOTAL_TRANSACTION_COUNT','DOWNTIME_IN_MINS','ATM_MAX_CAPACITY'], axis=1,inplace=True)

In [20]:
df.columns

Index(['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
       'DISPENSED_AMOUNT', 'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH'],
      dtype='object')

In [21]:
features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID',
    'HOUR', 'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR',
       'DAYOFYEAR', 'DAYOFMONTH']

In [22]:
print (df.shape)
df = df[df['DISPENSED_AMOUNT'] > 0]
print (df.shape)

(312417, 13)
(257564, 13)


In [23]:
#To be used for Experimentation
#df.to_csv('/data/Output/transaction_processed.csv',index=False)

In [24]:
X = df[features]
y = df['DISPENSED_AMOUNT']

In [25]:
X.head()

,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,ATM_ID,HOUR,DAYOFWEEK,QUARTER,MONTH,YEAR,DAYOFYEAR,DAYOFMONTH
0,1,0,3,18,299,0,6,1,1,2023,29,29
1,1,0,3,18,299,0,0,1,1,2023,30,30
2,1,0,3,18,299,0,1,1,1,2023,31,31
3,1,0,3,18,299,0,2,1,2,2023,32,1
4,1,0,3,18,299,0,3,1,2,2023,33,2


In [26]:
y

0         225180.0
1         262440.0
2         561150.0
3         437220.0
4         360900.0
            ...   
312412    195300.0
312413    245160.0
312414    222480.0
312415    198360.0
312416    314190.0
Name: DISPENSED_AMOUNT, Length: 257564, dtype: float64

In [27]:
# Split the data into training and test sets. (0.75, 0.25) split.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

In [28]:
print(f'Total # of sample in whole dataset: {len(X)}')
print("*****"*10)
print(f'Total # of sample in train dataset: {len(X_train)}')
print(f'Shape of X_train: {X_train.shape}')
print("*****"*10)
print(f'Total # of sample in test dataset: {len(X_test)}')
print(f'Shape of X_test: {X_test.shape}')

Total # of sample in whole dataset: 257564
**************************************************
Total # of sample in train dataset: 193173
Shape of X_train: (193173, 12)
**************************************************
Total # of sample in test dataset: 64391
Shape of X_test: (64391, 12)


# Decision Tree

In [29]:
tree = DecisionTreeRegressor(max_depth=4,max_features=4)

In [30]:
tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=4, max_features=4)

In [31]:
predictions = tree.predict(X_test)

In [32]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 193277.72660576506
Mean Squared Error: 60136801657.95109
Root Mean Squared Error: 245228.06050277175
r2_score: 0.13345750728022598


In [33]:
tree.feature_importances_

array([0.        , 0.        , 0.12827561, 0.03926546, 0.43079182,
       0.        , 0.        , 0.07454589, 0.00949336, 0.        ,
       0.010811  , 0.30681687])

In [34]:
pd.Series(tree.feature_importances_,index=features).sort_values(ascending=False)

ATM_ID          0.430792
DAYOFMONTH      0.306817
BANK_ID         0.128276
QUARTER         0.074546
STATE           0.039265
DAYOFYEAR       0.010811
MONTH           0.009493
HOLIDAY_FLAG    0.000000
WEEKDAY_FLAG    0.000000
HOUR            0.000000
DAYOFWEEK       0.000000
YEAR            0.000000
dtype: float64

In [35]:
param_grid = [{"max_depth":[3,4,5, None], "max_features":[3,4,5,6,7]}]

In [36]:
gs = GridSearchCV(estimator=DecisionTreeRegressor(random_state=123),param_grid = param_grid,cv=10)

In [37]:
gs.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=123),
             param_grid=[{'max_depth': [3, 4, 5, None],
                          'max_features': [3, 4, 5, 6, 7]}])

In [38]:
gs.cv_results_['params']

[{'max_depth': 3, 'max_features': 3},
 {'max_depth': 3, 'max_features': 4},
 {'max_depth': 3, 'max_features': 5},
 {'max_depth': 3, 'max_features': 6},
 {'max_depth': 3, 'max_features': 7},
 {'max_depth': 4, 'max_features': 3},
 {'max_depth': 4, 'max_features': 4},
 {'max_depth': 4, 'max_features': 5},
 {'max_depth': 4, 'max_features': 6},
 {'max_depth': 4, 'max_features': 7},
 {'max_depth': 5, 'max_features': 3},
 {'max_depth': 5, 'max_features': 4},
 {'max_depth': 5, 'max_features': 5},
 {'max_depth': 5, 'max_features': 6},
 {'max_depth': 5, 'max_features': 7},
 {'max_depth': None, 'max_features': 3},
 {'max_depth': None, 'max_features': 4},
 {'max_depth': None, 'max_features': 5},
 {'max_depth': None, 'max_features': 6},
 {'max_depth': None, 'max_features': 7}]

In [39]:
gs.cv_results_['rank_test_score']

array([18, 19, 15, 10, 20, 14, 17, 16,  7, 11, 13,  8, 12,  6,  9,  5,  4,
        3,  1,  2], dtype=int32)

In [40]:
gs.best_estimator_

DecisionTreeRegressor(max_features=6, random_state=123)

In [44]:
#Build Tree using Best parameters

tree1 = DecisionTreeRegressor(max_features=7, random_state=123)
tree1.fit(X_train, y_train)
predictions = tree1.predict(X_test)
y_pred = predictions

In [45]:
X_train.columns

Index(['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID', 'HOUR',
       'DAYOFWEEK', 'QUARTER', 'MONTH', 'YEAR', 'DAYOFYEAR', 'DAYOFMONTH'],
      dtype='object')

In [46]:
print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 3077.3148421363235
Mean Squared Error: 849635679.3930829
Root Mean Squared Error: 29148.51075772282
r2_score: 0.987757157028194


In [49]:
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)
y_pred_df = pd.DataFrame(y_pred, columns=['PREDICTION'])

In [51]:
type(X_train),type(X_test),type(y_train_df),type(y_test_df),type(y_pred_df)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [52]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=tree1, 
    session=my_session,
    x_train=X_train,
    y_train=y_train_df,
    x_test=X_test,
    y_test=y_test_df,
    y_pred=y_pred_df,
    source="Notebook",
    dataset_name="ATM_TRANSACTION_MASTER_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Dtree_Dispense_Amount_Regressor",
    description="ATM Dispense Amount DTree Regression",
    flavour="sklearn",
    model_type="regression",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function explained_variance_score.<locals>.explained_variance_score_anon_sproc at 0x7f1921acfa60>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████████                                                     25.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_887009E5_7B9A_4FDE_8EA6_AAED690C9D23_FDC_DTREE_DISPENSE_AMOUNT_REGRESSOR' registered successfully."

# 2nd Version of Dtree model

In [53]:
tree2 = DecisionTreeRegressor(max_features=8, random_state=123)
tree2.fit(X_train, y_train)
predictions = tree2.predict(X_test)
y_pred = predictions
y_prediction = pd.Series(y_pred)


print('Mean Absolute Error:', mean_absolute_error(y_test,predictions))
print('Mean Squared Error:', mean_squared_error(y_test,predictions))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test,predictions)))
print('r2_score:', r2_score(y_test,predictions))

Mean Absolute Error: 2012.7190135267351
Mean Squared Error: 629459630.5989968
Root Mean Squared Error: 25089.03407066515
r2_score: 0.9909297883770377


In [54]:
y_train_df = pd.DataFrame(y_train)
y_test_df = pd.DataFrame(y_test)
y_pred_df = pd.DataFrame(y_pred, columns=['PREDICTION'])

In [55]:
type(X_train),type(X_test),type(y_train_df),type(y_test_df),type(y_pred_df)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [56]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=tree2, 
    session=my_session,
    x_train=X_train,
    y_train=y_train_df,
    x_test=X_test,
    y_test=y_test_df,
    y_pred=y_pred_df,
    source="Notebook",
    dataset_name="ATM_TRANSACTION_MASTER_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Dtree_Dispense_Amount_Regressor",
    description="ATM Dispense Amount DTree Regression",
    flavour="sklearn",
    model_type="regression",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████████                                                     25.0%


Calculating build time metrics

Progress: ███████████████████████████████████                                    50.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████                   75.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_887009E5_7B9A_4FDE_8EA6_AAED690C9D23_FDC_DTREE_DISPENSE_AMOUNT_REGRESSOR' registered successfully."

# Use Snowflake Data for Model Prediction

In [54]:
table_name = 'ATM_TRANSACTION_MASTER_DATA'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [55]:
df = df[df['DISPENSED_AMOUNT'] > 0]

In [56]:
df.columns

Index(['DATE', 'WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'BANK_NAME',
       'ATM_ID', 'DISPENSED_AMOUNT', 'TOTAL_TRANSACTION_COUNT',
       'DOWNTIME_IN_MINS', 'ATM_MAX_CAPACITY', 'MALE_POPULATION',
       'FEMALE_POPULATION', 'TOTAL_POPULATION', 'NO_OF_VETERANS',
       'FOREIGN_BORN', 'AVG_HOUSEHOLD_SIZE', 'LANDAREA_SQ_MILE',
       'POPULATION_RATIO', 'POPULATION_DENSITY', 'MEDIAN_HOUSE_INCOME',
       'LOCATION_TYPE', 'ATM_PLACEMENTS', 'ATM_ACCESSIBILITY',
       'ATM_MAINTAINENANCE', 'ATM__REPLENISHMENT'],
      dtype='object')

In [57]:
df_train = df.sample(frac = 0.7)
df_test = df.drop(df_train.index)

In [58]:
df_train.shape, df_test.shape

((180295, 26), (77269, 26))

In [59]:
def get_prediction(data):
    df = data.copy()
    to_drop = ['MALE_POPULATION','FEMALE_POPULATION','TOTAL_POPULATION','NO_OF_VETERANS','FOREIGN_BORN','AVG_HOUSEHOLD_SIZE','LANDAREA_SQ_MILE','POPULATION_RATIO','POPULATION_DENSITY','MEDIAN_HOUSE_INCOME','LOCATION_TYPE','ATM_PLACEMENTS','ATM_ACCESSIBILITY','ATM_MAINTAINENANCE','ATM__REPLENISHMENT']
    df.drop(to_drop, inplace=True, axis=1)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['HOUR'] = df['DATE'].dt.hour
    df['DAYOFWEEK'] = df['DATE'].dt.dayofweek
    df['QUARTER'] = df['DATE'].dt.quarter
    df['MONTH'] = df['DATE'].dt.month
    df['YEAR'] = df['DATE'].dt.year
    df['DAYOFYEAR'] = df['DATE'].dt.dayofyear
    df['DAYOFMONTH'] = df['DATE'].dt.day
    df['DATE'] = pd.to_datetime(df['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
    
    df['WEEKDAY_FLAG'] = df['WEEKDAY_FLAG'].map({'Y': 1, 'N': 0})
    df['HOLIDAY_FLAG'] = df['HOLIDAY_FLAG'].map({'Y': 1, 'N': 0})
    
    df['STATE']= label_encoder.transform(df['STATE'])
    df['ATM_ID']= atm_encoder.transform(df['ATM_ID'])
    
    df.drop(['DATE','BANK_NAME'], axis=1,inplace=True)
    
    features = ['WEEKDAY_FLAG', 'HOLIDAY_FLAG', 'BANK_ID', 'STATE', 'ATM_ID','HOUR', 'DAYOFWEEK', 
                'QUARTER', 'MONTH', 'YEAR','DAYOFYEAR', 'DAYOFMONTH']
    
    X_feat = df[features]
    return tree1.predict(X_feat)

In [60]:
df_train_y  = get_prediction(df_train)
df_test_y  = get_prediction(df_test)

In [61]:
df_train['PREDICTED_DISPENSE_AMOUNT'] = df_train_y
df_test['PREDICTED_DISPENSE_AMOUNT'] = df_test_y

In [62]:
df_train['DATE'] = pd.to_datetime(df_train['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")
df_test['DATE'] = pd.to_datetime(df_test['DATE']).dt.strftime("%Y-%m-%d %H:%M:%S.%f")

In [63]:
df_train.head()

,DATE,WEEKDAY_FLAG,HOLIDAY_FLAG,BANK_ID,STATE,BANK_NAME,ATM_ID,DISPENSED_AMOUNT,TOTAL_TRANSACTION_COUNT,DOWNTIME_IN_MINS,...,LANDAREA_SQ_MILE,POPULATION_RATIO,POPULATION_DENSITY,MEDIAN_HOUSE_INCOME,LOCATION_TYPE,ATM_PLACEMENTS,ATM_ACCESSIBILITY,ATM_MAINTAINENANCE,ATM__REPLENISHMENT,PREDICTED_DISPENSE_AMOUNT
9069,2022-03-08 00:00:00.000000,Y,N,3,North Dakota,Citibank,TBH0002740218,211500.0,100.0,0.0,...,69001,0.50,2.746192,66519,Urban,Outdoor,24 by 7,Regular,Weekly,211500.0
148974,2021-06-29 00:00:00.000000,Y,N,1,North Carolina,Wells Fargo,APAN35711,13950.0,5.0,0.0,...,48618,0.48,62.943745,61972,Rural,Outdoor,24 by 7,Irregular,Daily,13950.0
218511,2021-01-30 00:00:00.000000,N,N,2,Washington,BNY Mellon,SPCNG392,273600.0,74.0,542.0,...,66456,0.50,37.620486,84247,Urban,Outdoor,24 by 7,Regular,Monthly,273600.0
32052,2022-12-09 00:00:00.000000,N,N,3,Oklahoma,Citibank,TBH0072520909,689940.0,239.0,0.0,...,68595,0.49,21.123916,55826,Rural,Indoor,24 by 7,Irregular,Daily,689940.0
275877,2021-08-25 00:00:00.000000,Y,N,3,Maryland,Citibank,TNY000660816,419130.0,119.0,70.0,...,9707,0.48,135.173483,90203,Sub-Urban,Drive-Thru,Restrcited hours,Regular,Monthly,419130.0


In [64]:
#df_train.to_csv('/data/Output/transaction_train_output.csv',index=True)
#df_test.to_csv('/data/Output/transaction_test_output.csv',index=True)

# Push the output to Snowflake tables

In [68]:
df_train_sf=my_session.createDataFrame(
        df_train.values.tolist(),
        schema=df_train.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT")

In [69]:
df_test_sf=my_session.createDataFrame(
        df_test.values.tolist(),
        schema=df_test.columns.tolist())
df_test_sf.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT")

# Run below query on snowflake to create REPORT_DATE

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT ADD COLUMN REPORT_DATE DATE

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET REPORT_DATE =  TO_DATE(DATE)

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT ADD COLUMN REPORT_DATE DATE

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET REPORT_DATE =  TO_DATE(DATE)

# Run below query on snowflake to UPDATE REPORT_DATE

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET REPORT_DATE =  DATEADD(MONTH, 14, REPORT_DATE)

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET REPORT_DATE =  DATEADD(MONTH, 14, REPORT_DATE)

# Run below query on snowflake to UPDATE DISPENSED_AMOUNT and PREDICTED_DISPENSED_AMOUNT
UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET DISPENSED_AMOUNT = ROUND(DISPENSED_AMOUNT/100)*10

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET PREDICTED_DISPENSE_AMOUNT = ROUND(PREDICTED_DISPENSE_AMOUNT/100)*10

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET DISPENSED_AMOUNT = ROUND(DISPENSED_AMOUNT/100)*10

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET PREDICTED_DISPENSE_AMOUNT = ROUND(PREDICTED_DISPENSE_AMOUNT/100)*10

# Run below query on Snowflake to ADD MEDIAN_HOUSING_INCOME

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT ADD COLUMN MEDIAN_HOUSING_INCOME DOUBLE

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TEST_OUTPUT
SET MEDIAN_HOUSING_INCOME =  TO_DOUBLE(MEDIAN_HOUSE_INCOME)

ALTER TABLE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT ADD COLUMN MEDIAN_HOUSING_INCOME DOUBLE

UPDATE FDC_BANKING_FS.PUBLIC.ATM_TRANSACTION_TRAIN_OUTPUT
SET MEDIAN_HOUSING_INCOME =  TO_DOUBLE(MEDIAN_HOUSE_INCOME)